In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [168]:
def distance(X, Y):
    (x1, x2) = X
    (y1, y2) = Y
    return ((x1-x2)**2 + (y1-y2)**2) ** 0.5

In [169]:
df_plays = pd.read_csv('plays.csv')
df_plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8552,2021110100,4310,(1:56) (Shotgun) P.Mahomes sacked at NYG 16 fo...,4,3,8,KC,NYG,NYG,8,...,NaN,18.0,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,"1 DL, 3 LB, 7 DB",SCRAMBLE,0,Bracket,Other
8553,2021110100,4363,(1:07) (Shotgun) D.Jones pass short right to E...,4,1,10,NYG,KC,NYG,25,...,NaN,35.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE,0,Cover-2,Zone
8554,2021110100,4392,"(1:01) (No Huddle, Shotgun) D.Jones sacked at ...",4,2,7,NYG,KC,NYG,28,...,NaN,38.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone
8555,2021110100,4411,"(:39) (No Huddle, Shotgun) D.Jones pass incomp...",4,3,15,NYG,KC,NYG,20,...,NaN,30.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone


In [170]:
df_players = pd.read_csv('players.csv')
df_players.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick


In [171]:
df_pffScouting = pd.read_csv('pffScoutingData.csv')
df_pffScouting.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [172]:
df_tracking = pd.DataFrame()
weeks = range(1, 9)

for week in weeks:
    file = 'week' + str(week) + '.csv'
    df_tracking_temp = pd.read_csv(file)
    df_tracking_temp['week'] = [week]*len(df_tracking_temp)
    df_tracking = pd.concat([df_tracking, df_tracking_temp]).reset_index(drop=True)

In [174]:
## Standardize tracking data such that orientation is the same going from left to right regardless of play dir

df_tracking.x = np.where(df_tracking.playDirection == 'left', 120 - df_tracking.x, df_tracking.x)
df_tracking.y = np.where(df_tracking.playDirection == 'left', 160/3  - df_tracking.y, df_tracking.y)
df_tracking.dir = np.where(df_tracking.playDirection == 'left', df_tracking.dir + 180, df_tracking.dir)
df_tracking.dir = np.where(df_tracking.dir > 360, df_tracking.dir - 360, df_tracking.dir)
df_tracking.o = np.where(df_tracking.playDirection == 'left', df_tracking.o + 180, df_tracking.o)
df_tracking.o = np.where(df_tracking.o > 360, df_tracking.o - 360, df_tracking.o)

In [175]:
los = df_tracking[(df_tracking.team == 'football') & (df_tracking.frameId == 1)][['gameId', 'playId', 'x']]
los = los.rename(columns = {'x': 'los'})

ball_data = df_tracking[(df_tracking.team == 'football')][['gameId', 'playId', 'frameId', 'x', 'y']]
ball_data = ball_data.rename(columns = {'x': 'ball_x', 'y': 'ball_y'})

df_tracking = df_tracking.merge(los, on = ['gameId', 'playId'], how = 'left')
df_tracking = df_tracking.merge(ball_data, on = ['gameId', 'playId', 'frameId'], how = 'left')

df_tracking['dist_from_los'] = abs(df_tracking.los - df_tracking.x)
df_tracking['euc_dist_from_ball'] = distance((df_tracking.x, df_tracking.y),
                                         (df_tracking.ball_x, df_tracking.ball_y))
df_tracking['y_dist_from_ball'] = abs(df_tracking.ball_y - df_tracking.y)
df_tracking['x_dist_from_ball'] = abs(df_tracking.ball_x - df_tracking.x)

In [176]:
first_frame = df_tracking[df_tracking.frameId == 1]
first_frame = first_frame.rename(columns = {'x': 'first_x', 'y': 'first_y'})

df_tracking = df_tracking.merge(first_frame[['gameId', 'playId', 'nflId', 'first_x', 'first_y']],
                                on = ['gameId', 'playId', 'nflId'], how = 'left')
df_tracking['x_delta'] = df_tracking.first_x - df_tracking.x
df_tracking['y_delta'] = df_tracking.first_y - df_tracking.y

In [177]:
## Creating the labels of pass rush or coverage

df_pffScouting['pass_rush'] = np.where(df_pffScouting.pff_role == 'Pass Rush', 1, 0)
df_pffScouting['coverage'] = np.where(df_pffScouting.pff_role == 'Coverage', 1, 0)


## Create more effective position labels
# positions = []
# for i in range(len(df_pffScouting.pff_positionLinedUp)):
#     pos = df_pffScouting.pff_positionLinedUp.iloc[i]
#     if pos in ['RLB', 'RILB', 'MLB', 'ILB', 'LILB', 'LLB']:
#         positions.append('LB')
#     elif pos in ['DE', 'LE', 'REO', 'LEO', 'OLB', 'ROLB', 'LOLB']:
#         positions.append('EDGE')
#     elif pos in ['FS', 'SS', 'FSR', 'SSR', 'FSL', 'SSL']:
#         positions.append('S')
#     elif pos in ['NT', 'DRT', 'DLT', 'NLT', 'NRT']:
#         positions.append('IDL')
#     elif pos in ['CB', 'RCB', 'LCB', 'SCBR', 'SCBoR', 'SCBiR', 'SCBL', 'SCBoL', 'SCBiL']:
#         positions.append('CB')
#     else:
#         positions.append(pos)
# df_pffScouting['position'] = positions

In [178]:
df_tracking = df_tracking.merge(df_players[['nflId', 'officialPosition']], on = 'nflId')

In [179]:
df_tracking = df_tracking.rename(columns = {'officialPosition': 'position'})
df_tracking

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,ball_y,dist_from_los,euc_dist_from_ball,y_dist_from_ball,x_dist_from_ball,first_x,first_y,x_delta,y_delta,position
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.220000,...,24.040000,5.15,23.239124,0.18,5.15,37.77,24.220000,0.00,0.00,QB
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.220000,...,23.910000,5.14,22.792455,0.31,4.45,37.77,24.220000,-0.01,0.00,QB
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.240000,...,23.890000,5.14,22.748407,0.35,4.39,37.77,24.220000,-0.01,-0.02,QB
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.250000,...,23.890000,5.19,22.656445,0.36,4.37,37.77,24.220000,0.04,-0.03,QB
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.260000,...,23.910000,5.23,22.386000,0.35,4.13,37.77,24.220000,0.08,-0.04,QB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952687,2021110100,3471,46772.0,76,2021-11-02T02:42:23.200,93.0,NYG,left,75.79,41.673333,...,5.153333,13.75,64.489960,36.52,15.91,62.97,33.103333,-12.82,-8.57,OLB
7952688,2021110100,3471,46772.0,77,2021-11-02T02:42:23.300,93.0,NYG,left,76.09,41.533333,...,4.593333,14.05,65.741579,36.94,15.57,62.97,33.103333,-13.12,-8.43,OLB
7952689,2021110100,3471,46772.0,78,2021-11-02T02:42:23.400,93.0,NYG,left,76.37,41.373333,...,4.053333,14.33,66.994199,37.32,15.19,62.97,33.103333,-13.40,-8.27,OLB
7952690,2021110100,3471,46772.0,79,2021-11-02T02:42:23.500,93.0,NYG,left,76.64,41.173333,...,3.523333,14.60,68.246259,37.65,14.81,62.97,33.103333,-13.67,-8.07,OLB


In [181]:
# Create the cache
dfs = {}

a = time.time()

# FOR EACH PLAY ADD THE RELEVANT TRACKING DATA INTO MEMORY
for x in range(len(df_plays)):
    play_id = df_plays.iloc[x]['playId']
    game_id = df_plays.iloc[x]['gameId']
    
    # Get the game tracking data
    gamedf = df_tracking[df_tracking.gameId == game_id]
    
    # Get the play tracking data within each game
    playdf = gamedf[gamedf.playId == play_id]
    
    # Data of moment where ball is snapped
    playdf_ball_snap = playdf[playdf.event == 'ball_snap']
    
    # Data of moment where pass is thrown or qb sack
    playdf_ball_thrown = playdf[(playdf.event == 'pass_forward') |
                                (playdf.event == 'qb_sack') | 
                                (playdf.event == 'run')]

    # Frame of when snap occurred
    frame_ball_snap = np.min(playdf_ball_snap.frameId)
    
    # Frame of when throw, sack, run, etc. occurred
    frame_ball_thrown = np.min(playdf_ball_thrown.frameId)
    
    # Frames pre-snap and after-snap
    playdf_pre_snap = playdf[playdf.frameId <= frame_ball_snap]
    
    # Ball data
    playdf_ball = playdf[playdf.team == 'football']
    
    # SAVE THE DATA INTO THE CACHE FOR EACH TIMEFRAME
    dfs[str(game_id) + '--BALL+' + str(play_id)] = playdf_ball
    dfs[str(game_id) + '--ball_snap+' + str(play_id)] = playdf_ball_snap
    dfs[str(game_id) + '--ball_thrown+' + str(play_id)] = playdf_ball_thrown
    dfs[str(game_id) + '--pre_snap+' + str(play_id)] = playdf_pre_snap
    
b = time.time()

print(b-a)

134.71928691864014


In [182]:
def distance(X, Y):
    (x1, y1) = X
    (x2, y2) = Y
    return ((x1-x2)**2 + (y1-y2)**2) ** 0.5

# offense_pos = set(['QB', 'RB', 'WR', 'TE', 'T', 'G', 'C', 'FB'])
# defense_pos = set(['CB', 'DE', 'DT', 'FS', 'MLB', 'OLB', 'SS'])

def nearest_player(player_input, tracking_dict):
    offense_pos = set(['QB', 'RB', 'WR', 'TE', 'T', 'G', 'C', 'FB'])
    defense_pos = set(['CB', 'DE', 'DT', 'FS', 'MLB', 'OLB', 'SS'])
    
    result = {}
    
    (pos, nflId, num) = player_input
    
    player_frames = tracking_dict[player_input]
    
    dist_nearest_off = []
    nearest_off_dir = []
    nearest_off_xy = []
    for i in range(len(player_frames)):
        min_off_p = []
        min_off_dir = []
        min_off_xy = []
        min_def_p = []
        min_def_xy = []
        for k, v in tracking_dict.items():
            if (k == (pos, nflId, num)): continue
            if (len(v) == 0): continue
            if (k[0] not in offense_pos): continue
            player = (player_frames[i][0], player_frames[i][1])
            other_player = (v[i][0], v[i][1])
            min_off_p.append(distance(player, other_player))
            min_off_dir.append(abs(player_frames[i][3]-v[i][3]))
            min_off_xy.append(other_player)
        dist_nearest_off.append(min(min_off_p))
        nearest_off_dir.append(min_off_dir[np.argmin(min_off_p)])
        nearest_off_xy.append(min_off_xy[np.argmin(min_off_p)])
            
    result['off_mean'] = np.mean(dist_nearest_off)
    result['off_var'] = np.var(dist_nearest_off)
    result['off_dir_mean'] = np.mean(nearest_off_dir)
    result['off_dir_var'] = np.var(nearest_off_dir)
    
    
    dist_nearest_def = []
    nearest_def_xy = []
    for i in range(len(player_frames)):
        min_off_p = []
        min_off_dir = []
        min_off_xy = []
        min_def_p = []
        min_def_xy = []
        for k, v in tracking_dict.items():
            if (k == (pos, nflId, num)): continue
            if (len(v) == 0): continue
            if (k[0] not in defense_pos): continue
            player = (player_frames[i][0], player_frames[i][1])
            other_player = (v[i][0], v[i][1])
            min_def_p.append(distance(player, other_player))
            min_def_xy.append(other_player)

        dist_nearest_def.append(min(min_def_p))
        nearest_def_xy.append(min_def_xy[np.argmin(min_def_p)])
    
    ratio_def = []
    ratio_def_off = []
    
    def_off_distances = []
    for x in range(len(nearest_def_xy)):
        dist = distance(nearest_off_xy[x],nearest_def_xy[x])
        def_off_distances.append(dist)

    for j in range(len(dist_nearest_def)):
        if def_off_distances[j] == 0:
            ratio_d = 0
        else:
            ratio_d = float(dist_nearest_def[j])/float(def_off_distances[j])
        ratio_def_off.append(ratio_d)
        
    result['def_off_rat_mean'] = np.mean(ratio_def_off)
    result['def_off_rat_var'] = np.var(ratio_def_off)
    result['def_off_rat_begin'] = ratio_def_off[0]
    result['def_off_rat_mid'] = ratio_def_off[(len(ratio_def_off)-1)//2]
    result['def_off_rat_snap'] = ratio_def_off[len(ratio_def_off)-1]
    
    if(len(result.keys()) == 0): print("EMPTY")
    return result

In [183]:
to_df = []


for x in range(len(df_plays)):
    play = df_plays.iloc[x]
    play_id = play['playId']
    game_id = play['gameId']

    df_ball = dfs[str(game_id) + '--BALL+' + str(play_id)]
    df_play = dfs[str(game_id) + '--pre_snap+' + str(play_id)]

    prev = None
    prev_num = None
    curr = []
    pos_dict = {}

    for index, row in df_play.iterrows():
        if(row['team'] == 'football'): continue
        if(row['nflId'] == prev):
            curr.append((row['x'],row['y'],row['s'],row['dir']))
        else:
            if(prev != None):
                position = (df_players[df_players['nflId'] == prev]).iloc[0]['officialPosition']
                pos_dict[(position,prev,prevNum)] = curr
            prev = row['nflId']
            prevNum = row['jerseyNumber']
            curr = []
            curr = [(row['x'],row['y'],row['s'],row['dir'])]
    
    for k , v in pos_dict.items(): 
        if(len(pos_dict[k]) == 0): continue
        featurerow = nearest_player(k,pos_dict)
        featurerow['gameId'] = game_id
        featurerow['playId'] = play_id
        featurerow['nflId'] = k[1]
        featurerow['number'] = k[2]
        to_df.append(featurerow)

nearest_df = pd.DataFrame(to_df)

In [184]:
nearest_df

,off_mean,off_var,off_dir_mean,off_dir_var,def_off_rat_mean,def_off_rat_var,def_off_rat_begin,def_off_rat_mid,def_off_rat_snap,gameId,playId,nflId,number
0,2.001243,0.000444,124.343333,4302.934156,0.879549,5.436777e-05,0.875826,0.869688,0.891289,2021090900,97,25511.0,12.0
1,2.460438,0.006679,165.783333,2052.407222,0.783788,1.692378e-03,0.866721,0.760112,0.739361,2021090900,97,35481.0,87.0
2,1.324893,0.125043,134.580000,219.101833,0.694815,1.636086e-03,0.750875,0.706580,0.632716,2021090900,97,35634.0,81.0
3,2.001243,0.000444,124.343333,4302.934156,0.961702,6.747537e-05,0.968265,0.970235,0.948344,2021090900,97,39985.0,25.0
4,1.538562,0.000059,41.853333,1329.163789,0.537962,1.196819e-05,0.537388,0.543792,0.532233,2021090900,97,40151.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179167,17.085947,0.010306,225.707143,8139.232249,1.586258,1.908631e-02,1.772462,1.604569,1.360769,2021110100,4433,40017.0,32.0
179168,2.941985,0.004127,124.912857,3177.556535,0.797814,3.033082e-06,0.797612,0.795099,0.797097,2021110100,4433,42406.0,55.0
179169,11.678842,0.158273,69.194286,37.044767,0.997210,1.382297e-04,0.979671,0.997277,1.014581,2021110100,4433,47984.0,27.0
179170,2.511668,0.000025,165.327143,4137.569306,0.783416,6.266329e-05,0.791790,0.784898,0.768750,2021110100,4433,44835.0,88.0


In [185]:
nearest_df = nearest_df[['off_mean', 'off_var', 'off_dir_mean', 'off_dir_var',
                         'def_off_rat_mean', 'def_off_rat_var', 'def_off_rat_begin',
                         'def_off_rat_mid', 'def_off_rat_snap',
                         'gameId', 'playId', 'nflId']]
df_tracking = df_tracking.merge(nearest_df, on = ['gameId', 'playId', 'nflId'])

In [186]:
df_tracking

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,position,off_mean,off_var,off_dir_mean,off_dir_var,def_off_rat_mean,def_off_rat_var,def_off_rat_begin,def_off_rat_mid,def_off_rat_snap
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,QB,2.001243,0.000444,124.343333,4302.934156,0.879549,0.000054,0.875826,0.869688,0.891289
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,...,QB,2.001243,0.000444,124.343333,4302.934156,0.879549,0.000054,0.875826,0.869688,0.891289
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,...,QB,2.001243,0.000444,124.343333,4302.934156,0.879549,0.000054,0.875826,0.869688,0.891289
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,...,QB,2.001243,0.000444,124.343333,4302.934156,0.879549,0.000054,0.875826,0.869688,0.891289
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,...,QB,2.001243,0.000444,124.343333,4302.934156,0.879549,0.000054,0.875826,0.869688,0.891289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7572049,2021103112,4167,45536.0,32,2021-11-01T03:24:38.700,10.0,DAL,right,80.62,24.10,...,QB,2.249272,0.000041,63.973333,13357.593456,0.916153,0.000010,0.912882,0.914405,0.921028
7572050,2021103112,4167,45536.0,33,2021-11-01T03:24:38.800,10.0,DAL,right,80.48,24.19,...,QB,2.249272,0.000041,63.973333,13357.593456,0.916153,0.000010,0.912882,0.914405,0.921028
7572051,2021103112,4167,45536.0,34,2021-11-01T03:24:38.900,10.0,DAL,right,80.33,24.28,...,QB,2.249272,0.000041,63.973333,13357.593456,0.916153,0.000010,0.912882,0.914405,0.921028
7572052,2021103112,4167,45536.0,35,2021-11-01T03:24:39.000,10.0,DAL,right,80.15,24.34,...,QB,2.249272,0.000041,63.973333,13357.593456,0.916153,0.000010,0.912882,0.914405,0.921028


In [ ]:
preSnap_tracking = pd.DataFrame()

for x in range(len(df_plays)):
    play_id = df_plays.iloc[x]['playId']
    game_id = df_plays.iloc[x]['gameId']
    
    # Get the game tracking data
    gamedf = df_tracking[df_tracking.gameId == game_id]
    
    # Get the play tracking data within each game
    playdf = gamedf[gamedf.playId == play_id]
    
    # Data of moment where ball is snapped
    playdf_ball_snap = playdf[playdf.event == 'ball_snap']

    # Frame of when snap occurred
    frame_ball_snap = np.min(playdf_ball_snap.frameId)
    
    # Frames pre-snap and after-snap
    playdf_pre_snap = playdf[playdf.frameId <= frame_ball_snap]
    
    playdf_pre_snap = playdf_pre_snap.reset_index(drop = True)
    preSnap_tracking = pd.concat([preSnap_tracking, playdf_pre_snap])
    preSnap_tracking = preSnap_tracking.reset_index(drop = True)

In [ ]:
preSnap_tracking

In [ ]:
# defensive_pos = ['CB', 'S', 'EDGE', 'LB', 'IDL']
defensive_pos = ['CB', 'DE', 'DT', 'FS', 'ILB', 'MLB', 'NT', 'OLB', 'SS']

num_weeks = 8

df_results = pd.DataFrame()

for position in defensive_pos:
    pos_df = preSnap_tracking[preSnap_tracking.position == position]
    for i in range(1, num_weeks+1):
        train_df = pos_df[pos_df.week != i]
        train_df = train_df[['dir', 'dist_from_los', 'y_dist_from_ball', 
                             'x_dist_from_ball', 'x_delta', 'y_delta', 
                             'off_mean', 'off_var', 'off_dir_mean', 'off_dir_var',
                             'def_off_rat_begin', 'def_off_rat_mid', 
                             'def_off_rat_snap', 'pass_rush']]
        train_df = train_df.dropna()
        train_x = train_df.drop(columns = 'pass_rush')
        train_y = train_df['pass_rush']
        
        
        test_df = pos_df[pos_df.week == i]
        test_df = test_df[['dir', 'dist_from_los', 'y_dist_from_ball', 
                             'x_dist_from_ball', 'x_delta', 'y_delta', 
                             'off_mean', 'off_var', 'off_dir_mean', 'off_dir_var',
                             'def_off_rat_begin', 'def_off_rat_mid', 
                             'def_off_rat_snap', 'pass_rush']]
        test_df = test_df.dropna()
        test_x = test_df.drop(columns = 'pass_rush')
        test_y = test_df['pass_rush']
        
        
        randForest = RandomForestClassifier(n_estimators = 500, max_depth = 5)
        randForest.fit(train_x, train_y.ravel())
        
        # Use predict_proba with randForest when predicting to get probabilistic estimate
        
        rf_preds = pd.DataFrame(np.round(randForest.predict_proba(test_x)[:,1], 3), 
                                columns = ['pass_rush_prob'])
        
        test_df = test_df.reset_index(drop = True)
        rf_preds = rf_preds.reset_index(drop = True)
        test_res = pd.concat([test_df, rf_preds], axis = 1)
        test_res['position'] = [position]*len(test_res)
        
        test_res = test_res.reset_index(drop = True)
        df_results = pd.concat([df_results, test_res])
        df_results = df_results.reset_index(drop = True)

In [ ]:
df_results

In [ ]:
print(df_results.shape)
df_res = df_results.dropna()
df_res.shape

feature_names = ['dir', 'dist_from_los', 'y_dist_from_ball', 
                             'x_dist_from_ball', 'x_delta', 'y_delta', 
                             'off_mean', 'off_var', 'off_dir_mean', 'off_dir_var',
                             'def_off_rat_begin', 'def_off_rat_mid', 
                             'def_off_rat_snap']

importances = randForest.feature_importances_
forest_importances = pd.Series(importances, index=feature_names)
std = np.std([tree.feature_importances_ for tree in randForest.estimators_], axis=0)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()